<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/NLP/Kaggle_movie_sentiment/Movies_review_sentiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare X_train, y_train

In [11]:
import pandas as pd
import numpy as np

In [12]:
train_path='/content/drive/My Drive/Data/NLP/Kaggle_movie_review_sentiment/train.tsv'
test_path='/content/drive/My Drive/Data/NLP/Kaggle_movie_review_sentiment/test.tsv'

In [13]:
train_df=pd.read_csv(train_path,sep='\t')
print(train_df.shape)
train_df.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [14]:
train_df['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [15]:
X=train_df['Phrase']
y=train_df['Sentiment']

In [16]:
X

0         A series of escapades demonstrating the adage ...
1         A series of escapades demonstrating the adage ...
2                                                  A series
3                                                         A
4                                                    series
                                ...                        
156055                                            Hearst 's
156056                            forced avuncular chortles
156057                                   avuncular chortles
156058                                            avuncular
156059                                             chortles
Name: Phrase, Length: 156060, dtype: object

In [17]:
y

0         1
1         2
2         2
3         2
4         2
         ..
156055    2
156056    1
156057    3
156058    2
156059    2
Name: Sentiment, Length: 156060, dtype: int64

In [18]:
test_df=pd.read_csv(test_path,sep='\t')
print(test_df.shape)
test_df.head()

(66292, 3)


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


Text have no label, so we split training set to train and text set

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,
                                               stratify=y,random_state=42)

In [23]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(109242,)
(109242,)
(46818,)
(46818,)


In [24]:
y_train.value_counts(normalize=True)

2    0.509950
3    0.210990
1    0.174759
4    0.058988
0    0.045312
Name: Sentiment, dtype: float64

In [25]:
y_test.value_counts(normalize=True)

2    0.509932
3    0.210987
1    0.174762
4    0.058994
0    0.045324
Name: Sentiment, dtype: float64

# Text Classification chap 8 Twitter_Sentiment analysis Orelilly NLP

In [ ]:
#!pip install text_normalizer

In [5]:
!pip install demoji

In [26]:
'''
Contains helper funtions for preprocessing of twitter documents before getting their corresponding vectors
from word2vec_twitter model
Author: Anuj Gupta
'''

'''
functionality:
Discard non-english tweets
Discard Replies
Discard RTs
For now this is handle while fetch tweet text from mongo documents
process constantbrands mentions
process any other mentions
process constant brand name if any 
process hanstags
process URLs
process websites
process process_EmailIds
process 
process alphanums
'''

import re
import string
import demoji
demoji.download_codes()
from nltk.tokenize import TweetTokenizer

#gobal
PunctChars = r'''[`'“".?!,:;]'''
Punct = '%s+' % PunctChars
Entity = '&(amp|lt|gt|quot);'
printable = set(string.printable)

# helper functoins
def regex_or(*items):
	r = '|'.join(items)
	r = '(' + r + ')'
	return r

def pos_lookahead(r):
	return '(?=' + r + ')'

def neg_lookahead(r):
	return '(?!' + r + ')'

def optional(r):
	return '(%s)?' % r

def trim(transient_tweet_text):
	''' 
	trim leading and trailing spaces in the tweet text
	'''
	return transient_tweet_text.strip()

def strip_whiteSpaces(transient_tweet_text):
	'''
	Strip all white spaces
	'''
	transient_tweet_text = re.sub(r'[\s]+', ' ', transient_tweet_text)
	return transient_tweet_text

def to_LowerCase(transient_tweet_text):
	'''
	Convert tweet text to lower to lower case alphabets
	'''
	transient_tweet_text = transient_tweet_text.lower()
	return transient_tweet_text

def prune_multple_consecutive_same_char(transient_tweet_text):
	'''
	yesssssssss  is converted to yess 
	ssssssssssh is converted to ssh
	'''
	transient_tweet_text = re.sub(r'(.)\1+', r'\1\1', transient_tweet_text)
	return transient_tweet_text

def remove_spl_words(transient_tweet_text):
	transient_tweet_text = transient_tweet_text.replace('&amp;',' and ')

	return transient_tweet_text

def strip_unicode(transient_tweet_text):
    '''
    Strip all unicode characters from a tweet
    '''
    tweet = ''.join(i for i in transient_tweet_text if ord(i)<128)
    return tweet 

def process_URLs(transient_tweet_text):
	'''
	replace all URLs in the tweet text
	'''
	UrlStart1 = regex_or('https?://', r'www\.',r'bit.ly/')
	CommonTLDs = regex_or('com','co\\.uk','org','net','info','ca','biz','info','edu','in','au')
	UrlStart2 = r'[a-z0-9\.-]+?' + r'\.' + CommonTLDs + pos_lookahead(r'[/ \W\b]')
	UrlBody = r'[^ \t\r\n<>]*?'  # * not + for case of:  "go to bla.com." -- don't want period
	UrlExtraCrapBeforeEnd = '%s+?' % regex_or(PunctChars, Entity)
	UrlEnd = regex_or( r'\.\.+', r'[<>]', r'\s', '$')
	Url = 	(optional(r'\b') + 
    		regex_or(UrlStart1, UrlStart2) + 
    		UrlBody + 
    pos_lookahead( optional(UrlExtraCrapBeforeEnd) + UrlEnd))

	Url_RE = re.compile("(%s)" % Url, re.U|re.I)
	transient_tweet_text = re.sub(Url_RE, " constanturl ", transient_tweet_text)

	# fix to handle unicodes in URL
	URL_regex2 = r'\b(htt)[p\:\/]*([\\x\\u][a-z0-9]*)*'
	transient_tweet_text = re.sub(URL_regex2, " constanturl ", transient_tweet_text)
	return transient_tweet_text

def process_Websites(transient_tweet_text):
	'''
	identify website mentioned if any 
	'''
	CommonTLDs = regex_or('com','co\\.uk','org','net','info','ca','biz','info','edu','in','au')
	sep = r'[.]'
	website_regex = r'(?<!#)?(\b)?[a-zA-Z0-9.]+' + sep + CommonTLDs
	website_RE = re.compile("(%s)" % website_regex, re.U|re.I)
	transient_tweet_text = re.sub(website_RE, ' constantwebsite ', transient_tweet_text)

	return transient_tweet_text

def process_EmailIds(transient_tweet_text):
	'''
	identify email mentioned if any
	'''
	email_regex = r'(\b)?[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(\b)?'
	transient_tweet_text = re.sub(email_regex, ' constantemailid ', transient_tweet_text)

	return transient_tweet_text

def process_Mentions(transient_tweet_text):
	'''
	Identify mentions if any
	'''
	transient_tweet_text = re.sub(r"@(\w+)", " constantnonbrandmention ", transient_tweet_text)
	return transient_tweet_text
def process_HashTags(transient_tweet_text):
	'''
	Strip all Hashtags from a tweet
	'''
	transient_tweet_text = re.sub(r"#(\w+)\b", ' constanthashtag ', transient_tweet_text)
	return transient_tweet_text

def process_Dates(transient_tweet_text):
	'''
	Identify date and convert it to constant
	'''
	#transient_tweet_text = re.sub(r'(\d+/\d+/\d+)', " constantdate " , transient_tweet_text)
	#transient_tweet_text = re.sub(r'constantnum[\s]?(/|-)[\s]?constantnum[\s]?(/|-)[\s]?constantnum', " constantdate " , transient_tweet_text)
	#date_regex = r'(constantnum)[\s]*(st|nd|rd|th)[\s]*(january|jan|february|feb|march|mar|april|may|june|jun|july|august|aug|september|sep|october|oct|november|nov|december|dec)'
	date_regex1 = r'\b((0|1|2|3)?[0-9][\s]*)[-./]([\s]*([012]?[0-9])[\s]*)([-./]([\s]*(19|20)[0-9][0-9]))?\b'
	transient_tweet_text = re.sub(date_regex1, ' constantdate ' , transient_tweet_text)
	date_regex2 = r'\b((19|20)[0-9][0-9][\s]*[-./]?)?[\s]*([012]?[0-9])[\s]*[-./][\s]*(0|1|2|3)?[0-9]\b'
	transient_tweet_text = re.sub(date_regex2, ' constantdate ' , transient_tweet_text)

	Months = regex_or('january','jan','february','feb','march','mar','april','may','june','jun','july','jul','august','aug','september','sep','october','oct','november','nov','december','dec')
	date_regex3 = r'\d+[\s]*(st|nd|rd|th)[\s]*' + Months 
	transient_tweet_text = re.sub(date_regex3, ' constantdate ', transient_tweet_text)
	date_regex4 = Months + r'[\s]*\d+[\s]*(st|nd|rd|th)*\b'
	transient_tweet_text = re.sub(date_regex4, ' constantdate ' , transient_tweet_text)
	date_regex5 = r'[\s]?\b(19|20)[0-9][0-9]\b[\s]?'
	transient_tweet_text = re.sub(date_regex5, ' constantdate ' , transient_tweet_text)

	date_regex6 = r'([\s]*(constantdate))+'
	transient_tweet_text = re.sub(date_regex6, ' constantdate ' , transient_tweet_text)

	return transient_tweet_text

def process_Times(transient_tweet_text):
	'''
	Indentify time and convert it to constant
	'''
	time_regex1 = r'([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9][\s]*(am|pm)?[\s]*([iescm](st)|gmt|utc|[pmce](dt))?'
	transient_tweet_text = re.sub(time_regex1, ' constanttime ' , transient_tweet_text)

	return transient_tweet_text

def process_BrandMentions(transient_tweet_text):
	'''
	process all airrwoot brands Mentions if any in tweet text
	'''
	constant_brands = regex_or('paytmcare', 'paytm','snapdeal_help','snapdeal','bluestone_com','shopohelp','shopo','mobikwikswat',
		'mobikwik','taxiforsure','tfscares','zoomcarindia','freshmenuindia','freshmenucares','grofers','jetairways',
		'cleartrip','olacabs','support_ola','makemytrip','makemytripcare','chai_point')

	BrandMentionRegex = r'@(\b)*' + constant_brands
	transient_tweet_text = re.sub(BrandMentionRegex, ' constantbrandmention ', transient_tweet_text)
	return transient_tweet_text

def process_NonBrandMentions(transient_tweet_text):
	'''
	process all Mentions left, if any, in tweet text
	'''
	transient_tweet_text = re.sub(r"@(\w+)", ' constantnonbrandmention ', transient_tweet_text)
	return transient_tweet_text

def process_BrandName(transient_tweet_text):
	'''
	process all airrwoot brands Mentions if any in tweet text
	'''
	constant_brands = regex_or('paytmcare', 'paytm','snapdeal_help','snapdeal','bluestone_com', 'bluestone','shopohelp','shopo',
		'mobikwikswat','mobikwik','taxiforsure','tfscares', 'tfs','zoomcarindia', 'zoomcar','freshmenuindia','freshmenucares', 
		'freshmenu','grofers','jetairways', 'jet','cleartrip','olacabs','support_ola', 'olacab', 'ola' ,'makemytrip',
		'makemytripcare', 'chai_point')

	BrandNameRegex = constant_brands
	transient_tweet_text = re.sub(BrandNameRegex, ' constantbrandname ', transient_tweet_text)
	return transient_tweet_text

def identify_Savings(transient_tweet_text):
	'''
	identify sale/save offers
	'''
	#sale_regex = r'(?<!#)\b(discount|discounts|sale|save|saver|super[\s]*saver|super[\s]*save)\b[\s]*(constantnum)*[\s]*[%]?[\s]*(-|~)?[\s]*(constantnum)*[\s]*[%]?'
	sale_regex = r'(?<!#)\b(discount|discounts|sale|save|saver|super[\s]*saver|super[\s]*save)\b[\s]*((rs|\$)*[\s]*(constantnum))*[\s]*[%]?[\s]*(-|~|or)?[\s]*((rs|\$)*[\s]*(constantnum))*[\s]*[%]?'
	transient_tweet_text = re.sub(sale_regex, " constantdiscount ", transient_tweet_text)
	#discount_List = []
	#discount_List = re.findall(r'constantdiscount', transient_tweet_text)
	return transient_tweet_text

def indentify_Offers(transient_tweet_text):
	'''
	identify cashbacks and off / substrings of the form "30% off" or "30% cashback" or "$30 off"
	Replace them by constantOFFER
	'''
	#transient_tweet_text = re.sub(r'[rs|$]?[ ]*[constantnum][ ]*[%]?[ ]?[off|cashback|offer]', "constantoffer", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:(up[\s]?to)?((rs|\$)*[\s]*(constantnum))[\s]*[%]?)?[\s]*[-|~|or]?[\.]?[\s]*((rs|\$)*[\s]*(constantnum))*[\s]*[%]?[\s]*(offer|off|cashback|cash|cash back)', " constantoffer ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:cashback|cash back|cash)\b', " constantoffer ", transient_tweet_text)
	#Offer_List = []
	#Offer_List = re.findall(r'constantoffer', transient_tweet_text)
	return transient_tweet_text

def indentify_Promos(transient_tweet_text):
	'''
	indentify coupons/promos with promo codes
	Assumption - promo code can be alphanumeric. But it immediately follows text of promo/code/promocode etc
	'''
	#transient_tweet_text = re.sub(r'\b(promocode|promo code|promo|code)[s]?[\s]*[a-z]*(constantnum)*[a-z]*[\s]+', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code)\b[\s]*(constantalphanum)\b', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code)\b[\s]*[a-z]+\b', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code)\b[\s]*[0-9]+\b', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code|coupon)[s]?\b', " constantpromo ", transient_tweet_text)
	#Promo_List = []
	#Promo_List = re.findall(r'constantpromo', transient_tweet_text)
	return transient_tweet_text

def indentify_Money(transient_tweet_text):
	'''
	identify money in the tweet text but outside offers. This includes $,Rs, pound, Euro
	'''
	money_regex1 = r'\b(rs|\$)[\s]*(constantnum)?[\.]?[\s]*constantnum\b'
	transient_tweet_text = re.sub(money_regex1, " constantmoney ", transient_tweet_text)
	money_regex2 = r'[\s]*[\.]?[\s]*constantnum(cent[s]?|\$|c)\b'
	transient_tweet_text = re.sub(money_regex2, " constantmoney ", transient_tweet_text)
	money_regex3 = r'(\$|rs)[\s]*constantalphanum'
	transient_tweet_text = re.sub(money_regex3, " constantmoney ", transient_tweet_text)
	#Money_List = []
	#Money_List = re.findall(r'constantmoney', transient_tweet_text)
	return transient_tweet_text

def indentify_freebies(transient_tweet_text):
	'''
	indentify freebies in tweets if any - free offers, free shipping, free trial, 
	'''
	freebies_regex1 = r'(?<!#)\b(?:free)[\s]+[a-z]+\b'
	transient_tweet_text = re.sub(freebies_regex1, " constantfreebies ", transient_tweet_text)
	freebies_regex2 = r'(?<!#)\b(?:free)[\s]+[a-z]*\b'
	transient_tweet_text = re.sub(freebies_regex2, " constantfreebies ", transient_tweet_text)
	return transient_tweet_text

def replace_numbers(transient_tweet_text):
	'''
	Given any number/interger in tweet text, we want it to be replaced by constantnum
	'''
	# we want to process only those numbers that are not in a hashtag - below logic does this
	num_regex = r'(?<!#)\b(?:[-+]?[\d,]*[\.]?[\d,]*[\d]+|\d+)\b'
	transient_tweet_text = re.sub(num_regex, " constantnum " , transient_tweet_text)
	return transient_tweet_text

def identify_AlphaNumerics(transient_tweet_text):
	'''
	Identify alpha numerics - this helps in identifying product codes/models, promocodes, Order IDs
	'''
	AlphaNumeric_regex = r'(?<!#)\b(?:([a-z]+[0-9]+[a-z]*|[a-z]*[0-9]+[a-z]+)[a-z,0-9]*)\b'
	transient_tweet_text = re.sub(AlphaNumeric_regex, " constantalphanum ", transient_tweet_text)
	return transient_tweet_text

def strip_whiteSpaces(transient_tweet_text):
	'''
	Strip all white spaces
	'''
	transient_tweet_text = re.sub(r'[\s]+', ' ', transient_tweet_text)
	return transient_tweet_text

def prune_multple_consecutive_same_char(transient_tweet_text):
	'''
	yesssssssss  is converted to yess 
	ssssssssssh is converted to ssh
	'''
	transient_tweet_text = re.sub(r'(.)\1+', r'\1\1', transient_tweet_text)
	return transient_tweet_text

def remove_spl_words(transient_tweet_text):
	transient_tweet_text = transient_tweet_text.replace('&amp;',' and ')

	return transient_tweet_text

def remove_emoji(transient_tweet_text):
    tweet_tokenizer = TweetTokenizer()
    tokenized_tweet = tweet_tokenizer.tokenize(transient_tweet_text)
    emojis_present = demoji.findall(transient_tweet_text)
    tweet_no_emoji=''
    for i,s in enumerate(tokenized_tweet):
        if s in emojis_present.keys():
            tweet_no_emoji = tweet_no_emoji + ' ' + emojis_present[s]
        else:
            tweet_no_emoji = tweet_no_emoji + ' ' + s
    return tweet_no_emoji

def deEmojify(transient_tweet_text):
    return transient_tweet_text.encode('ascii', 'ignore').decode('ascii')

# def print_test():
    
#     test_tweet = "Nice @varun paytm @paytm saver abc@gmail.com sizes for the wolf on 20/10/2010 at 10:00PM  grey/deep royal-volt Nike Air Skylon II retro are 40% OFF for a limited time at $59.99 + FREE shipping.BUY HERE -> https://bit.ly/2L2n7rB (promotion - use code MEMDAYSV at checkout)"
#     #General Proprocessing
#     test_tweet = test_tweet.lower()
#     test_tweet = strip_unicode(test_tweet)
    
#     #function tests
#     print("Process URLS:\n",process_URLs(test_tweet))
#     print("Remove websites:\n",process_Websites(test_tweet))
#     print("Remove mentions:\n",process_Mentions(test_tweet))
#     print('Remove Emailid:\n',process_EmailIds(test_tweet))
#     print("Remove Hashtags:\n",process_HashTags(test_tweet))
#     print("Remove Dates:\n",process_Dates(test_tweet))
#     print("Process Time:\n",process_Times(test_tweet))
#     print("Process Brand Mention:\n",process_BrandMentions(test_tweet))
#     print("Process non Brand Mention:\n",process_NonBrandMentions(test_tweet))
#     print("Process Brand Name:\n",process_BrandName(test_tweet))
#     print("Process Savings:\n",identify_Savings(test_tweet))
#     print("Process Offers:\n",indentify_Offers(test_tweet))
#     print("Identiy Promos:\n",indentify_Promos(test_tweet))
# ############
# print_test()




def process_TweetText(tweet_text):
	'''
	Takes tweet_text and preprocesses it 
	Order of preprocessing:
	'''

	# get utf-8 encoding, lowercase, trim and remove multiple white spaces
	transient_tweet_text = tweet_text
	transient_tweet_text = strip_unicode(transient_tweet_text)
	#print "PROCESSED: ", transient_tweet_text

	transient_tweet_text = to_LowerCase(transient_tweet_text)
	transient_tweet_text = trim(transient_tweet_text)
	transient_tweet_text = strip_whiteSpaces(transient_tweet_text)
	transient_tweet_text = remove_spl_words(transient_tweet_text)

 
	#emoji
	transient_tweet_text = remove_emoji(transient_tweet_text) 
	transient_tweet_text = deEmojify(transient_tweet_text)
	# process Hastags, URLs, Websites, process_EmailIds
	# Give precedence to url over hashtag
	transient_tweet_text = process_URLs(transient_tweet_text)
	transient_tweet_text = process_HashTags(transient_tweet_text)
	#transient_tweet_text = process_Websites(transient_tweet_text)
	transient_tweet_text = process_EmailIds(transient_tweet_text)

	# process for brand mention, any other mention and brand Name
	#transient_tweet_text = process_BrandMentions(transient_tweet_text)
	#transient_tweet_text = process_NonBrandMentions(transient_tweet_text)
	transient_tweet_text = process_Mentions(transient_tweet_text)
	#transient_tweet_text = process_BrandName(transient_tweet_text)

	# remove any unicodes
	transient_tweet_text = strip_unicode(transient_tweet_text)

	# identify Date / Time if any
	transient_tweet_text = process_Times(transient_tweet_text)
	transient_tweet_text = process_Dates(transient_tweet_text)

	# indentify alphanums and nums
	transient_tweet_text = identify_AlphaNumerics(transient_tweet_text)
	transient_tweet_text = replace_numbers(transient_tweet_text)
	
	# identify promos, savings, offers, money and freebies
	transient_tweet_text = indentify_Promos(transient_tweet_text)
	transient_tweet_text = identify_Savings(transient_tweet_text)
	transient_tweet_text = indentify_Offers(transient_tweet_text)
	transient_tweet_text = indentify_Money(transient_tweet_text)
	transient_tweet_text = indentify_freebies(transient_tweet_text)

	transient_tweet_text = trim(transient_tweet_text)
	transient_tweet_text = strip_whiteSpaces(transient_tweet_text)

	transient_tweet_text = prune_multple_consecutive_same_char(transient_tweet_text)

	return transient_tweet_text

# if __name__ == "__main__":
# print(process_TweetText("Nice @varun paytm @paytm saver abc@gmail.com sizes for the wolf on 20/10/2010 at 10:00PM  grey/deep royal-volt Nike Air Skylon II retro are 40% OFF for a limited time at $59.99 + FREE shipping.BUY HERE -> https://bit.ly/2L2n7rB (promotion - use code MEMDAYSV at checkout)"))

... OK (Got response in 0.26 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [27]:
#Making the necessary imports
import os
import sys

#preprocessing_path = "/home/etherealenvy/Downloads/practical-nlp/Ch8/O5_smtd_preprocessing.py"
#sys.path.append(os.path.abspath(preprocessing_path))

#import O5_smtd_preprocessing

from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords')

import pandas as pd
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()


#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
X_train_df=pd.DataFrame(X_train)
X_train_df

,Phrase
150238,make you wish you were at home watching that m...
133360,", the tale has turned from sweet to bitterswee..."
49191,can say that about most of the flicks moving i...
137709,"Does n't deliver a great story ,"
73297,unrecoverable life
...,...
142974,"Laced with liberal doses of dark humor , gorge..."
78231,Offers absolutely nothing I had n't already se...
7375,Ivan is a prince of a fellow
115288,the stomach-knotting suspense


In [29]:
X_train_df['Phrase'].values

array(['make you wish you were at home watching that movie instead of in the theater watching this one',
       ', the tale has turned from sweet to bittersweet , and when the tears come during that final , beautiful scene , they finally feel absolutely earned .',
       'can say that about most of the flicks moving in and out of the multiplex',
       ..., 'Ivan is a prince of a fellow',
       'the stomach-knotting suspense', 'Quirky'], dtype=object)

In [30]:
y_train

150238    1
133360    4
49191     2
137709    1
73297     1
         ..
142974    4
78231     1
7375      3
115288    3
2467      2
Name: Sentiment, Length: 109242, dtype: int64

In [31]:
X_train_df['Phrase_process'] = X_train_df['Phrase'].apply(lambda x: process_TweetText(x))

In [32]:
X_train_df.head()

,Phrase,Phrase_process
150238,make you wish you were at home watching that m...,make you wish you were at home watching that m...
133360,", the tale has turned from sweet to bitterswee...",", the tale has turned from sweet to bitterswee..."
49191,can say that about most of the flicks moving i...,can say that about most of the flicks moving i...
137709,"Does n't deliver a great story ,","does n't deliver a great story ,"
73297,unrecoverable life,unrecoverable life


In [31]:
X_train_df['phrase_tokens'] = X_train_df['Phrase_process'].apply(lambda x: tweet_tokenizer.tokenize(x))
X_train_df['phrase_no_stopwords'] = X_train_df['phrase_tokens'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])

In [32]:
X_train_df.head()

,Phrase,Phrase_process,phrase_tokens,phrase_no_stopwords
150238,make you wish you were at home watching that m...,make you wish you were at home watching that m...,"[make, you, wish, you, were, at, home, watchin...","[make, wish, home, watching, movie, instead, t..."
133360,", the tale has turned from sweet to bitterswee...",", the tale has turned from sweet to bitterswee...","[,, the, tale, has, turned, from, sweet, to, b...","[,, tale, turned, sweet, bittersweet, ,, tears..."
49191,can say that about most of the flicks moving i...,can say that about most of the flicks moving i...,"[can, say, that, about, most, of, the, flicks,...","[say, flicks, moving, multiplex]"
137709,"Does n't deliver a great story ,","does n't deliver a great story ,","[does, n't, deliver, a, great, story, ,]","[n't, deliver, great, story, ,]"
73297,unrecoverable life,unrecoverable life,"[unrecoverable, life]","[unrecoverable, life]"


## Train your own Embedding

In [33]:
phrase_processed=X_train_df['Phrase_process'].values

In [35]:
#CBOW
import time
start = time.time()
w2v_model = Word2Vec(phrase_processed,min_count=5, sg=0)
end = time.time()
print("CBOW Model Training Complete.\nTime taken for training is:{:.5f} sec ".format((end-start)))

CBOW Model Training Complete.
Time taken for training is:7.45960 sec 


In [37]:
#Create document vectors by averaging word vectors.
def embedding_feats(list_of_lists):
    DIMENSION = 100
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        feats.append(feat_for_this/count_for_this if count_for_this > 0 else feat_for_this)        
    return feats

train_vectors = embedding_feats(X_train_df['phrase_no_stopwords'].values)
print(len(train_vectors))

109242


In [40]:
#Take any classifier (LogisticRegression here)
classifier = LogisticRegression(random_state=2020)
classifier.fit(train_vectors, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2020, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
X_test_df=pd.DataFrame(X_test)
X_test_df['Phrase_process'] = X_test_df['Phrase'].apply(lambda x: process_TweetText(x))
X_test_df['phrase_tokens'] = X_test_df['Phrase_process'].apply(lambda x: tweet_tokenizer.tokenize(x))
X_test_df['phrase_no_stopwords'] = X_test_df['phrase_tokens'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
X_test_df.head()


,Phrase,Phrase_process,phrase_tokens,phrase_no_stopwords
105687,"seems just as expectant of an adoring , wide-s...","seems just as expectant of an adoring , wide-s...","[seems, just, as, expectant, of, an, adoring, ...","[seems, expectant, adoring, ,, wide-smiling, r..."
133291,"the dream world of teen life ,","the dream world of teen life ,","[the, dream, world, of, teen, life, ,]","[dream, world, teen, life, ,]"
36948,purpose and finesse,purpose and finesse,"[purpose, and, finesse]","[purpose, finesse]"
101915,"Argento , at only 26","argento , at only constantnum","[argento, ,, at, only, constantnum]","[argento, ,, constantnum]"
150567,you may ask,you may ask,"[you, may, ask]","[may, ask]"


In [42]:
phrase_processed=X_test_df['Phrase_process'].values
#CBOW
import time
start = time.time()
w2v_model = Word2Vec(phrase_processed,min_count=5, sg=0)
end = time.time()
print("CBOW Model Training Complete.\nTime taken for training is:{:.5f} sec ".format((end-start)))

CBOW Model Training Complete.
Time taken for training is:3.24631 sec 


In [43]:
test_vectors = embedding_feats(X_test_df['phrase_no_stopwords'].values)
print(len(test_vectors))

46818


In [44]:
preds = classifier.predict(test_vectors)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.10      0.13      0.12      2122
           1       0.19      0.13      0.15      8182
           2       0.53      0.84      0.65     23874
           3       0.18      0.02      0.03      9878
           4       0.00      0.00      0.00      2762

    accuracy                           0.46     46818
   macro avg       0.20      0.22      0.19     46818
weighted avg       0.35      0.46      0.37     46818



# Deep learning (Chap 4 classification Oreillly NLP)

## Prepare data

In [33]:

#Make the necessary imports
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential
from keras.initializers import Constant

In [34]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

In [35]:
train_path='/content/drive/My Drive/Data/NLP/Kaggle_movie_review_sentiment/train.tsv'
test_path='/content/drive/My Drive/Data/NLP/Kaggle_movie_review_sentiment/test.tsv'

In [36]:
train_df=pd.read_csv(train_path,sep='\t')
print(train_df.shape)
train_df.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [63]:
X=train_df['Phrase']
y=train_df['Sentiment']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,
                                               stratify=y,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(109242,)
(109242,)
(46818,)
(46818,)


## 1. Tokenize the texts and convert them into word index vectors

In [38]:
X_train.values

array(['make you wish you were at home watching that movie instead of in the theater watching this one',
       ', the tale has turned from sweet to bittersweet , and when the tears come during that final , beautiful scene , they finally feel absolutely earned .',
       'can say that about most of the flicks moving in and out of the multiplex',
       ..., 'Ivan is a prince of a fellow',
       'the stomach-knotting suspense', 'Quirky'], dtype=object)

In [39]:
#Vectorize these text samples into a 2D integer tensor using Keras Tokenizer
#Tokenizer is fit on training data only, and that is used to tokenize both train and test data.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS) #20000
tokenizer.fit_on_texts(X_train.values)
train_sequences = tokenizer.texts_to_sequences(X_train.values) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(X_test.values)
word_index = tokenizer.word_index #(X_train, luc nay chua phan ra X_train_new va X_valid)
print('Found %s unique tokens.' % len(word_index))
# max 20000 words keep in sequences, word index is just a show case, not use (> max words)

Found 15264 unique tokens.


In [40]:
print(train_sequences[0]) # max 10000 words in sequences
print(len(train_sequences[0]))

[77, 22, 970, 22, 175, 30, 448, 227, 9, 17, 378, 3, 7, 1, 317, 227, 18, 26]
18



## 2. Pad the text sequences so that all text vectors are of the same length.

In [41]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
#initial padding of 0s, until vector is of size MAX_SEQUENCE_LENGTH
train_sequences_padding = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH) #1000
test_sequences_padding = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH) #1000

In [42]:
#Before transform
print(len(train_sequences)) # train_texts --> train_sequences
print(len(train_sequences[0]))
print(train_sequences[0])

109242
18
[77, 22, 970, 22, 175, 30, 448, 227, 9, 17, 378, 3, 7, 1, 317, 227, 18, 26]


In [43]:
#After transform
print(train_sequences_padding.shape)
train_sequences_padding

(109242, 1000)


array([[   0,    0,    0, ...,  227,   18,   26],
       [   0,    0,    0, ...,  140, 1487, 3657],
       [   0,    0,    0, ...,    3,    1, 4473],
       ...,
       [   0,    0,    0, ...,    3,    2, 2915],
       [   0,    0,    0, ..., 1308, 9589,  436],
       [   0,    0,    0, ...,    0,    0,  429]], dtype=int32)

In [44]:
train_labels = to_categorical(np.asarray(y_train))
test_labels = to_categorical(np.asarray(y_test))

print(y_train[:3])
print(train_labels.shape)
train_labels

150238    1
133360    4
49191     2
Name: Sentiment, dtype: int64
(109242, 5)


array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

## Split the training data into a training set and a validation set

In [45]:
X_train_new,X_valid,y_train_new,y_valid=train_test_split(train_sequences_padding,train_labels,
                                                         test_size=0.3,
                                               stratify=train_labels,random_state=42)
print(X_train_new.shape)
print(y_train_new.shape)
print(X_valid.shape)
print(y_valid.shape)

(76469, 1000)
(76469, 5)
(32773, 1000)
(32773, 5)


##3. Map every word index to an embedding vector (for pre-train model| can skip if no need pre train)

In [46]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open('/content/drive/My Drive/Data/NLP/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0] # vocabulary name
        coefs = np.asarray(values[1:], dtype='float32') # embedding 100 dim
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
print(embeddings_index["google"])
print(len(embeddings_index["google"]))
print(MAX_NUM_WORDS)
print(EMBEDDING_DIM)

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.
[ 0.22575  -0.56253  -0.05156  -0.079389  1.1876   -0.48397  -0.23342
 -0.85278   0.97495  -0.33344   0.71692   0.12644   0.31962  -1.4136
 -0.57903  -0.037286 -0.0164    0.45155  -0.29005   0.52599  -0.22534
 -0.29556  -0.032407  1.5608   -0.013499 -0.064558  0.26625   0.78595
 -0.71693  -0.93025   0.80461   1.6035   -0.30602  -0.34764   0.93872
  0.38137  -0.26743  -0.56519   0.58899  -0.14554  -0.34324   0.21291
 -0.39887   0.090042 -0.8495    0.38803  -0.5045   -0.22488   1.0644
 -0.2624    1.0334    0.06348  -0.39989   0.24236  -0.65636  -1.8107
 -0.061801  0.13795   1.1658   -0.30046  -0.50143   0.16509   0.039835
  0.62541   0.56935   0.64125   0.21308   0.30276   0.39673   0.38973
  0.28183   0.79481  -0.11962  -0.49598  -0.53195  -0.14897   0.51254
 -0.39208  -0.58535  -0.078509  0.81721  -0.73497  -0.68131   0.099243
 -0.87608   0.029632  0.33402  -0.14305   0.16964  -0.035178  0.39777
  0.71769   0.25

In [47]:
# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
print(num_words)
print(embedding_matrix.shape)
print(len(word_index)) # max 20000 tu hay gap nhat thoi, index nay khong xai het
print(len(embeddings_index))

15265
(15265, 100)
15264
400000


In [48]:
for word, i in word_index.items():
    if i > MAX_NUM_WORDS: # chi thuc hien get embedding word < max_num_words(20000)
        continue
    embedding_vector = embeddings_index.get(word) #lay 100d embedding cho tung tu
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [49]:
print(embedding_matrix.shape)
embedding_matrix[1]

(15265, 100)


array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

In [64]:
labels_index=len(np.unique(y))
labels_index

5

## Build model

## Keras

In [57]:
# y_test = test_sequences_padding

1D CNN Model with pre-trained embedding

In [27]:
# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(input_dim=num_words,#15265 (max 20000)
                            output_dim=EMBEDDING_DIM, #100
                            embeddings_initializer=Constant(embedding_matrix), #(15265, 100)
                            input_length=MAX_SEQUENCE_LENGTH, #1000
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing of embedding matrix is done


In [28]:
labels_index=len(np.unique(y))
labels_index

5

In [29]:
print('Define a 1D CNN model.')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(labels_index, activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
cnnmodel.summary()

Define a 1D CNN model.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 100)         1526500   
_________________________________________________________________
conv1d (Conv1D)              (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 128)           82048     
_________________________________________________________________
global_max_pooling1d (Global (N

In [30]:
#Train the model. Tune to validation set. 
cnnmodel.fit(X_train_new, y_train_new,
          batch_size=200,
          epochs=1, validation_data=(X_valid, y_valid))

383/383 [==============================] - 22s 59ms/step - loss: 1.1060 - acc: 0.5578 - val_loss: 1.0369 - val_acc: 0.5784


In [31]:
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_sequences_padding, test_labels)
print('Test accuracy with CNN:', acc)

1464/1464 [==============================] - 10s 7ms/step - loss: 1.0420 - acc: 0.5764
Test accuracy with CNN: 0.5764449834823608


In [45]:
y_pred=cnnmodel.predict_classes(test_sequences_padding)
y_pred

array([2, 2, 2, ..., 1, 2, 2])

In [46]:
test_labels_reverse=np.argmax(test_labels, axis =1)
test_labels_reverse

array([1, 2, 3, ..., 3, 2, 2])

In [48]:
print(classification_report(test_labels_reverse,y_pred))

              precision    recall  f1-score   support

           0       0.28      0.35      0.31      2122
           1       0.39      0.26      0.31      8182
           2       0.65      0.85      0.74     23874
           3       0.51      0.36      0.42      9878
           4       0.66      0.09      0.15      2762

    accuracy                           0.58     46818
   macro avg       0.50      0.38      0.39     46818
weighted avg       0.56      0.58      0.54     46818



1D CNN model with training your own embedding¶

In [49]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

In [50]:

print("Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings")
cnnmodel2 = Sequential()
cnnmodel2.add(Embedding(MAX_NUM_WORDS, 128)) # co the thieu input length = MAX_SEQUENCE_LENGTH (1000)
cnnmodel2.add(Conv1D(128, 5, activation='relu'))
cnnmodel2.add(MaxPooling1D(5))
cnnmodel2.add(Conv1D(128, 5, activation='relu'))
cnnmodel2.add(MaxPooling1D(5))
cnnmodel2.add(Conv1D(128, 5, activation='relu'))
cnnmodel2.add(GlobalMaxPooling1D())
cnnmodel2.add(Dense(128, activation='relu'))
cnnmodel2.add(Dense(labels_index, activation='softmax'))

cnnmodel2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print(cnnmodel2.summary())

Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         82048     
_________

In [51]:
#Train the model. Tune to validation set. 
cnnmodel2.fit(X_train_new, y_train_new,
          batch_size=200, #75000 X_train shape
          epochs=1, validation_data=(X_valid, y_valid))

383/383 [==============================] - 41s 107ms/step - loss: 1.1858 - acc: 0.5329 - val_loss: 1.0873 - val_acc: 0.5795


In [52]:
#Evaluate on test set:
y_pred=cnnmodel2.predict_classes(test_sequences_padding)
test_labels_reverse=np.argmax(test_labels, axis =1)
print(classification_report(test_labels_reverse,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2122
           1       0.40      0.22      0.29      8182
           2       0.63      0.87      0.73     23874
           3       0.47      0.40      0.43      9878
           4       0.49      0.15      0.23      2762

    accuracy                           0.58     46818
   macro avg       0.40      0.33      0.34     46818
weighted avg       0.52      0.58      0.53     46818



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM Model with training your own embedding

In [53]:
print("Defining and training an LSTM model, training embedding layer on the fly")

#model
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(labels_index, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
rnnmodel.summary()

Defining and training an LSTM model, training embedding layer on the fly
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 2,692,229
Trainable params: 2,692,229
Non-trainable params: 0
_________________________________________________________________


In [54]:
#Train the model. Tune to validation set. 
rnnmodel.fit(X_train_new, y_train_new,
          batch_size=200, #75000 X_train shape
          epochs=1, validation_data=(X_valid, y_valid))
#  1472s 4s/step - loss: 0.3740 - accuracy: 0.5546 - val_loss: 0.3232 - val_accuracy: 0.6187
#Evaluate on test set:
y_pred=rnnmodel.predict_classes(test_sequences_padding)
test_labels_reverse=np.argmax(test_labels, axis =1)
print(classification_report(test_labels_reverse,y_pred))

383/383 [==============================] - 1472s 4s/step - loss: 0.3740 - accuracy: 0.5546 - val_loss: 0.3232 - val_accuracy: 0.6187
              precision    recall  f1-score   support

           0       0.43      0.01      0.01      2122
           1       0.50      0.38      0.43      8182
           2       0.70      0.83      0.76     23874
           3       0.49      0.59      0.53      9878
           4       0.59      0.05      0.09      2762

    accuracy                           0.62     46818
   macro avg       0.54      0.37      0.37     46818
weighted avg       0.60      0.62      0.58     46818



LSTM Model using pre-trained Embedding Layer

In [55]:

print("Defining and training an LSTM model, using pre-trained embedding layer")

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(labels_index, activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(rnnmodel2.summary())

Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 100)         1526500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total params: 1,644,393
Trainable params: 117,893
Non-trainable params: 1,526,500
_________________________________________________________________
None


In [56]:
#Train the model. Tune to validation set. 
rnnmodel2.fit(X_train_new, y_train_new,
          batch_size=200, #75000 X_train shape
          epochs=1, validation_data=(X_valid, y_valid))
#Evaluate on test set:
y_pred=rnnmodel2.predict_classes(test_sequences_padding)
test_labels_reverse=np.argmax(test_labels, axis =1)
print(classification_report(test_labels_reverse,y_pred))

383/383 [==============================] - 1357s 4s/step - loss: 0.3639 - accuracy: 0.5577 - val_loss: 0.3349 - val_accuracy: 0.5898
              precision    recall  f1-score   support

           0       0.38      0.01      0.02      2122
           1       0.44      0.27      0.33      8182
           2       0.67      0.84      0.74     23874
           3       0.46      0.53      0.49      9878
           4       0.52      0.06      0.11      2762

    accuracy                           0.59     46818
   macro avg       0.49      0.34      0.34     46818
weighted avg       0.56      0.59      0.55     46818



- DL model is better than ML model. (improve 20% macro and weighted avg, 10% accuracy)
- RNN model is better than CNN model, but very long time training (~ 20 times lower), accuracy is not imporve so much (just 4%)

# Pytorch

LSTM pytorch

In [18]:
#library imports
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
torch.manual_seed(0)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential
from keras.initializers import Constant
import pandas as pd
import numpy as np
import re
from collections import Counter
import string
from sklearn.metrics import mean_squared_error,accuracy_score

In [19]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

In [20]:
train_path='/content/drive/My Drive/Data/NLP/Kaggle_movie_review_sentiment/train.tsv'
test_path='/content/drive/My Drive/Data/NLP/Kaggle_movie_review_sentiment/test.tsv'

In [21]:
train_df=pd.read_csv(train_path,sep='\t')
print(train_df.shape)
train_df.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [22]:
X=train_df['Phrase']
y=train_df['Sentiment']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,
                                               stratify=y,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(109242,)
(109242,)
(46818,)
(46818,)


In [23]:
#Vectorize these text samples into a 2D integer tensor using Keras Tokenizer
#Tokenizer is fit on training data only, and that is used to tokenize both train and test data.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS) #20000
tokenizer.fit_on_texts(X_train.values)
train_sequences = tokenizer.texts_to_sequences(X_train.values) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(X_test.values)
word_index = tokenizer.word_index #(X_train, luc nay chua phan ra X_train_new va X_valid)
print('Found %s unique tokens.' % len(word_index))
# max 20000 words keep in sequences, word index is just a show case, not use (> max words)

Found 15264 unique tokens.


In [24]:
print(train_sequences[0]) # max 10000 words in sequences
print(len(train_sequences[0]))

[77, 22, 970, 22, 175, 30, 448, 227, 9, 17, 378, 3, 7, 1, 317, 227, 18, 26]
18


In [25]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
#initial padding of 0s, until vector is of size MAX_SEQUENCE_LENGTH
train_sequences_padding = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH) #1000
test_sequences_padding = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH) #1000

In [26]:
#Before transform
print(len(train_sequences)) # train_texts --> train_sequences
print(len(train_sequences[0]))
print(train_sequences[0])

109242
18
[77, 22, 970, 22, 175, 30, 448, 227, 9, 17, 378, 3, 7, 1, 317, 227, 18, 26]


In [27]:
#After transform
print(train_sequences_padding.shape)
train_sequences_padding

(109242, 1000)


array([[   0,    0,    0, ...,  227,   18,   26],
       [   0,    0,    0, ...,  140, 1487, 3657],
       [   0,    0,    0, ...,    3,    1, 4473],
       ...,
       [   0,    0,    0, ...,    3,    2, 2915],
       [   0,    0,    0, ..., 1308, 9589,  436],
       [   0,    0,    0, ...,    0,    0,  429]], dtype=int32)

In [28]:
train_labels = to_categorical(np.asarray(y_train))
test_labels = to_categorical(np.asarray(y_test))

print(y_train[:3])
print(train_labels.shape)
train_labels

150238    1
133360    4
49191     2
Name: Sentiment, dtype: int64
(109242, 5)


array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [29]:
X_train_new,X_valid,y_train_new,y_valid=train_test_split(train_sequences_padding,train_labels,
                                                         test_size=0.3,
                                               stratify=train_labels,random_state=42)
print(X_train_new.shape)
print(y_train_new.shape)
print(X_valid.shape)
print(y_valid.shape)

(76469, 1000)
(76469, 5)
(32773, 1000)
(32773, 5)


In [30]:
X_test=test_sequences_padding
y_test=test_labels
print(X_test.shape)
print(y_test.shape)

(46818, 1000)
(46818, 5)


In [31]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open('/content/drive/My Drive/Data/NLP/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0] # vocabulary name
        coefs = np.asarray(values[1:], dtype='float32') # embedding 100 dim
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
print(embeddings_index["google"])
print(len(embeddings_index["google"]))
print(MAX_NUM_WORDS)
print(EMBEDDING_DIM)

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.
[ 0.22575  -0.56253  -0.05156  -0.079389  1.1876   -0.48397  -0.23342
 -0.85278   0.97495  -0.33344   0.71692   0.12644   0.31962  -1.4136
 -0.57903  -0.037286 -0.0164    0.45155  -0.29005   0.52599  -0.22534
 -0.29556  -0.032407  1.5608   -0.013499 -0.064558  0.26625   0.78595
 -0.71693  -0.93025   0.80461   1.6035   -0.30602  -0.34764   0.93872
  0.38137  -0.26743  -0.56519   0.58899  -0.14554  -0.34324   0.21291
 -0.39887   0.090042 -0.8495    0.38803  -0.5045   -0.22488   1.0644
 -0.2624    1.0334    0.06348  -0.39989   0.24236  -0.65636  -1.8107
 -0.061801  0.13795   1.1658   -0.30046  -0.50143   0.16509   0.039835
  0.62541   0.56935   0.64125   0.21308   0.30276   0.39673   0.38973
  0.28183   0.79481  -0.11962  -0.49598  -0.53195  -0.14897   0.51254
 -0.39208  -0.58535  -0.078509  0.81721  -0.73497  -0.68131   0.099243
 -0.87608   0.029632  0.33402  -0.14305   0.16964  -0.035178  0.39777
  0.71769   0.25

In [32]:
# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
print(num_words)
print(embedding_matrix.shape)
print(len(word_index)) # max 20000 tu hay gap nhat thoi, index nay khong xai het
print(len(embeddings_index))

15265
(15265, 100)
15264
400000


In [33]:
for word, i in word_index.items():
    if i > MAX_NUM_WORDS: # chi thuc hien get embedding word < max_num_words(20000)
        continue
    embedding_vector = embeddings_index.get(word) #lay 100d embedding cho tung tu
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [34]:
print(embedding_matrix.shape)
embedding_matrix[1]

(15265, 100)


array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

In [35]:
labels_index=len(np.unique(y))
labels_index

5

### Model

In [36]:
# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(input_dim=num_words,#15265 (max 20000)
                            output_dim=EMBEDDING_DIM, #100
                            embeddings_initializer=Constant(embedding_matrix), #(15265, 100)
                            input_length=MAX_SEQUENCE_LENGTH, #1000
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing of embedding matrix is done


In [37]:
labels_index

5

In [38]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, n_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, n_layers, batch_first=True)
        self.output = nn.Linear(hidden_size, labels_index)
        
    def forward(self, x):
        out = self.embedding(x)
        out, _ = self.lstm(out)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.output(out)
        out = out[-1,0]
        out = torch.sigmoid(out).unsqueeze(0)
        
        return out

In [47]:
model = LSTM(MAX_NUM_WORDS, 100, 128, 1)
model

LSTM(
  (embedding): Embedding(20000, 100)
  (lstm): LSTM(100, 128, batch_first=True)
  (output): Linear(in_features=128, out_features=5, bias=True)
)

In [50]:
loss_function = nn.CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 5

In [51]:
losses = []
acc = []
for e in range(1, epochs+1):
    single_loss = []
    preds = []
    targets = []
    for i, r in enumerate(X_train_new):
      if len(r) <= 1:
          continue
      x = torch.Tensor([r]).long()
      y = torch.Tensor([y_train_new[i]])
      print('x',x)
      print('y',y)
      pred = model(x)
      print('pred',pred)
      loss = loss_function(pred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      final_pred = np.round(pred.detach().numpy())
      preds.append(final_pred)
      targets.append(y)
      single_loss.append(loss.item())
        
    losses.append(np.mean(single_loss))
    accuracy = accuracy_score(targets,preds)
    acc.append(accuracy)
    if e%1 == 0:
        print("Epoch: ", e, "... Loss function: ", losses[-1], "... Accuracy: ", acc[-1])

x tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,

RuntimeError: ignored

In [ ]:
x_range = range(len(losses))
plt.plot(x_range, losses)
plt.xlabel("epochs")
plt.ylabel("Loss function")
plt.show()

In [ ]:
x_range = range(len(acc))
plt.plot(x_range, acc)
plt.xlabel("epochs")
plt.ylabel("Accuracy score")
plt.show()